In [1]:
import torch
import torchvision
from torchvision import models
import torchvision.transforms as T
import time
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow
import cv2
print('pytorch', torch.__version__)
print('torchvision', torchvision.__version__)

pytorch 1.7.0+cu101
torchvision 0.8.1+cu101


In [2]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [4]:
IMG_SIZE = 480

COLORS = np.array([
    (0, 0, 0),       # 0=background
    (128, 0, 0),     # 1=aeroplane
    (0, 128, 0),     # 2=bicycle
    (128, 128, 0),   # 3=bird
    (0, 0, 128),     # 4=boat
    (128, 0, 128),   # 5=bottle
    (0, 128, 128),   # 6=bus
    (128, 128, 128), # 7=car
    (255, 255, 255), # 8=cat
    (192, 0, 0),     # 9=chair
    (64, 128, 0),    # 10=cow
    (192, 128, 0),   # 11=dining table
    (64, 0, 128),    # 12=dog
    (192, 0, 128),   # 13=horse
    (64, 128, 128),  # 14=motorbike
    (192, 128, 128), # 15=person
    (0, 64, 0),      # 16=potted plant
    (128, 64, 0),    # 17=sheep
    (0, 192, 0),     # 18=sofa
    (128, 192, 0),   # 19=train
    (0, 64, 128)     # 20=tv/monitor
])

In [5]:
deeplab = models.segmentation.deeplabv3_resnet101(pretrained=True).eval().to(device)

In [6]:
trf = T.Compose([
    T.Resize(IMG_SIZE),
#     T.CenterCrop(IMG_SIZE), # make square image
    T.ToTensor(), 
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])
# 4차원 텐서로 변경
# input_img = trf(img).unsqueeze(0)
# print(input_img)

In [12]:
fps = 30
fcc = cv2.VideoWriter_fourcc('M','J','P','G')
out_vid = cv2.VideoWriter('./DIVX.avi', fcc, fps,(480,480))
index = 0

In [ ]:
video = cv2.VideoCapture("./testvid.mp4")
assert video.isOpened()
nof_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
# video 읽기
while True:
    t = time.time()
    ret, frame = video.read()
    if not ret:
        break
    # if frame is None:
    #     break
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    im_pil = Image.fromarray(img)
    input_img = trf(im_pil).unsqueeze(0).to(device)
    out = deeplab(input_img)['out']
    out = torch.argmax(out.squeeze(), dim=0)
    out = out.detach().cpu().numpy()

    out_seg = seg_map(out)
    # out_seg = cv2.cvtColor(out_seg,cv2.COLOR_RGB2BGR)
    cv2_imshow(out_seg)
    # print(out_seg)
    out_vid.write(out_seg)
    
    fps = 1. / (time.time() - t)
    print('\rframe: % 4d / %d - framerate: %f fps ' % (index, nof_frames - 1, fps), end='')
    index = index + 1  
out_vid.release()

In [8]:
# 해당 인덱스에 해당하는 색을 채움
def seg_map(img, n_classes=21):
    rgb = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)

    for c in range(n_classes):
        idx = img == c

        rgb[idx] = COLORS[c]

    return rgb

In [ ]:
out_seg = seg_map(out)
print(out_seg)
fig, ax = plt.subplots(figsize=(16, 16))
ax.imshow(out_seg)